![](https://pbs.twimg.com/media/EmpjIGjXEAAtqXK.jpg:large)

# Motivation

Hi kagglers,

i hope you are doing well in this pandemic time. This is Going to be my 2nd Kaggle Notebook of this year and again i am going to use pytorch tpu for cassava leaf disease classification problem.in my last tpu notebook of this competition [ViT - Pytorch xla (TPU) for leaf disease](https://www.kaggle.com/mobassir/vit-pytorch-xla-tpu-for-leaf-disease?scriptVersionId=51452003)  i tried to convert awesome gpu baseline of [Kun Hao Yeh](https://www.kaggle.com/khyeh0719) into tpu and tried to describe each code cell thoroughly so that anyone can understand but this notebook work is going to be my personal favorite one,why?

Because In this Notebook i will try to convert @haqishen san's  magnificent [Baseline - Modified From Previous Competition](https://www.kaggle.com/haqishen/baseline-modified-from-previous-competition) which is a very good boilerplate to use in pretty much all future projects, so thank you [Qishen Ha](https://www.kaggle.com/haqishen) san and thank you for all the wisdom you shared, the main reason why i think this notebook work is going to be my personal favorite because we all know that this competition is very similar to last pandas challenge where we had label noise and in that competition my team used this notebook of qishen ha san [Train EfficientNet-B0 w/ 36 tiles_256 [LB0.87]](https://www.kaggle.com/haqishen/train-efficientnet-b0-w-36-tiles-256-lb0-87) and missed silver by only 1 place in private lb :( we tried a lot of different things and lot of different models using that baseline of qishen ha san which helped me get into 51st private lb position. we trained all models on rtx 2080ti and didn't use TPU back then in pandas competition so here in this kernel i am planning to convert that baseline into TPU for faster experiments. i hope it will help those who don't have personal GPU,love pytorch like me,want to do more experiments by investing less time(compared to GPU training) and also for beginners like me who want to learn how to make TPU baseline for a given data problem.

![](https://cdn-ak.f.st-hatena.com/images/fotolife/t/tereka/20200330/20200330235102.png)

# ChangeLog


* v1 : trying tf_efficientnet_lite4 for first fold only and for 5 epochs
* v2 : forgot to select TPU hardware in version 1,fixing that issue
* v3 : training fold 1 for 15 epochs without using any scheduler(failed)
* v4 : saving weight after each and every epoch,going for 10 epoch single fold for quick check if it fails again or not(failed)
* v5 : saving weight at the end of fold and using tpu 1.7 version,using scheduler
* v6 : AdamW and gradual warmup scheduler
* v7 : trying vit_large_patch16_384 (instead of efficientnet lite models) for image_size = 384 and epochs = 10(failed)
* v8 : 5 epoch(1 epoch takes only 10 minutes). don't know why kernel failing to finish commit for more epochs when i use xm.save(),if anybody knows the solution then please let me know
* v9 : running for 10 epoch and trying to save best weight file again inside epoch loop
* v10 : **bi tempered logistic loss** (failed)
* v11 : reducing batch size
* v12 : Fancy augmentation in train phase
* v13 : [TaylorCE loss + Label Smoothing Combo](https://www.kaggle.com/yerramvarun/cassava-taylorce-loss-label-smoothing-combo) related   [DISCUSSION](https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/209782) 
* v14 : smoothing 0.2
* v15 : tried normalization using datasets mean and std and it made cv worst somehow,using bi tempered logistic loss and more augmentation
* v16 : error fix
* v17 : got cv 0.896 on first fold, so now training  other 4 folds with same setting
* v18 : v17 failed because of SIGKILL (DON'T know the reason),now training fold 1 only
* v19 : fold 2
* v20 : fold 3
* v21 : fold 4 (failed)
* v22 : fold 4
* v23 : **fold 3** had lowest validation accuracy,trying to improve that using  Finetuning original model by freezing few layers as shown [here](https://www.kaggle.com/piantic/how-to-finetuning-models-pytorch-xla-tpu) and also applying scheduler after 4th epoch
* v24 :  scheduler for every epoch (commit stuck)
* v25 : retrying(stopped)
* v26 :  SymmetricCrossEntropy loss
* v27 :  fixing bug in train_model() function,added more augmentations,switching back to bi_tempered_logistic_loss
* v28 :  it seems in version 27 i got OOM,posted about it [here](https://www.kaggle.com/discussion/214063) , now reducing batch size and retrying (oom again)
* v29 : reducing augmentation and epoch  
* v30 : fixing syntaxError
* v31 : vit_base_patch16_384 long training

# install  torch xla nightly

In [ ]:

!python3.7 -m pip install --upgrade pip
!pip install torch==1.7.0
!pip install torchvision 
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl



In [ ]:
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version "nightly"

# Installing Gradual Warmup Scheduler

In [ ]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

# Imports

In [ ]:
import os

import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.utils.serialization as xser
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils

import sys; 

package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    '../input/image-fmix/FMix-master'
]
for pth in package_paths:
    sys.path.append(pth)
    

import torch

import warnings

import pandas as pd
import numpy as np
import torch.nn as nn

from sklearn.model_selection import train_test_split

from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageFile

import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler



import gc

import random

import skimage.io

from PIL import Image
import scipy as sp

import sklearn.metrics
from sklearn.metrics import accuracy_score

from functools import partial

from torch.utils.data import DataLoader, Dataset
import torchvision.models as models

from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2


from contextlib import contextmanager
from pathlib import Path
from collections import defaultdict, Counter



from fmix import sample_mask, make_low_freq_image, binarise_mask

from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io

from torch import nn

from datetime import datetime
import time
import random


from torchvision import transforms

from tqdm import tqdm

import matplotlib.pyplot as plt

from torch.utils.data.sampler import SequentialSampler, RandomSampler

from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics


import pydicom

from scipy.ndimage.interpolation import zoom


import PIL.Image

import torch.nn.functional as F

from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from warmup_scheduler import GradualWarmupScheduler

import albumentations

from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm_notebook as tqdm
from pylab import rcParams

#from efficientnet_pytorch import EfficientNet

warnings.filterwarnings("ignore")

# version of torch

In [ ]:
print(torch.__version__)

# For parallelization in TPUs

In [ ]:
os.environ["XLA_USE_BF16"] = "1"
os.environ["XLA_TENSOR_ALLOCATOR_MAXSIZE"] = "100000000"

# Config

In [ ]:
DEBUG = 0

data_dir = '../input/cassava-leaf-disease-classification'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
image_folder = os.path.join(data_dir, 'train_images')

kernel_type = 'vit_base_patch16_384' 

net_type = 'vit_base_patch16_384'
folds = 5
#fold = 0
image_size = 384
batch_size = 6
num_workers = 4
out_dim = 5
init_lr = 1e-4
warmup_factor = 7
warmup_epo = 1

smoothing = 0.3
t1 = 0.8
t2 = 1.4
freeze=True


n_epochs = 1 if DEBUG else 40
df_train = df_train.sample(500).reset_index(drop=True) if DEBUG else df_train

#device = torch.device('cuda')

print(image_folder)

# Create Folds

we will use StratifiedKFold

In [ ]:
skf = StratifiedKFold(folds, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['label'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

# Dataset

Custom dataset class that will process our dataset,it returns images and corresponding labels as torch tensor.

In [ ]:

class LEAFDataset(Dataset):
    def __init__(self, df, transforms=None):

        self.df = df.reset_index(drop=True)
        self.transforms = transforms

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        image_file = os.path.join(image_folder, img_id)
        image = cv2.imread(image_file)
        image = image[:, :, ::-1]

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        image = image.astype(np.float32)
        image /= 255
        image = image.transpose(2, 0, 1)

        return torch.tensor(image), torch.tensor(row.label)


# Augmentations

horizontal flip and resize for training and only resize for validation

In [ ]:
transforms_train = albumentations.Compose([
    #torchvision.transforms.Resize((image_size1, image_size)),
    albumentations.Resize(image_size, image_size),
    
    #albumentations.HorizontalFlip(p=0.5),
    albumentations.Flip(always_apply=True), # Either Horizontal, Vertical or both flips
   
    albumentations.Transpose(p=0.5),
    #albumentations.VerticalFlip(p=0.5),
    #albumentations.augmentations.transforms.ColorJitter(brightness=0.10, contrast=0.2, saturation=0.2, hue=0.00, always_apply=False, p=0.5),
    #albumentations.ShiftScaleRotate(p=0.5),
    #albumentations.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
    #albumentations.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    #albumentations.augmentations.transforms.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
    albumentations.CoarseDropout(max_holes=12, max_height=4, max_width=4, always_apply=False, p=0.5),
    #CustomCutout(p=1),
    #albumentations.Cutout(num_holes=5, max_h_size=24, max_w_size=24, fill_value=0, always_apply=False, p=0.5),
    #albumentations.Normalize(mean=[0.4303, 0.4967, 0.3134], std=[0.2142, 0.2191, 0.1954], max_pixel_value=255.0, p=1.0),
    
])
transforms_val = albumentations.Compose([
    #torchvision.transforms.Resize((image_size1, image_size)),
    albumentations.Resize(image_size, image_size),

    #albumentations.Normalize(mean=[0.4303, 0.4967, 0.3134], std=[0.2142, 0.2191, 0.1954], max_pixel_value=255.0, p=1.0),
])

# Visualizing dataset

In [ ]:
dataset_show = LEAFDataset(df_train, transforms=transforms_train)
rcParams['figure.figsize'] = 20,10
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        axarr[p].imshow(img.transpose(0, 1).transpose(1,2).squeeze())
        axarr[p].set_title(str(label))

# Model

In [ ]:
class cassavamodel(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        #print(self.model)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x
    

In [ ]:
''' 
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_name('efficientnet-b2')

#model.avg_pool = nn.AdaptiveAvgPool2d(1)
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, out_dim)
'''

# Loss Function

we'll use [**bi tempered logistic loss**](https://arxiv.org/pdf/1906.03361.pdf)

In [ ]:
# Code taken from https://github.com/fhopfmueller/bi-tempered-loss-pytorch/blob/master/bi_tempered_loss_pytorch.py

def log_t(u, t):
    """Compute log_t for `u'."""
    if t==1.0:
        return u.log()
    else:
        return (u.pow(1.0 - t) - 1.0) / (1.0 - t)

def exp_t(u, t):
    """Compute exp_t for `u'."""
    if t==1:
        return u.exp()
    else:
        return (1.0 + (1.0-t)*u).relu().pow(1.0 / (1.0 - t))

def compute_normalization_fixed_point(activations, t, num_iters):

    """Returns the normalization value for each example (t > 1.0).
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature 2 (> 1.0 for tail heaviness).
      num_iters: Number of iterations to run the method.
    Return: A tensor of same shape as activation with the last dimension being 1.
    """
    mu, _ = torch.max(activations, -1, keepdim=True)
    normalized_activations_step_0 = activations - mu

    normalized_activations = normalized_activations_step_0

    for _ in range(num_iters):
        logt_partition = torch.sum(
                exp_t(normalized_activations, t), -1, keepdim=True)
        normalized_activations = normalized_activations_step_0 * \
                logt_partition.pow(1.0-t)

    logt_partition = torch.sum(
            exp_t(normalized_activations, t), -1, keepdim=True)
    normalization_constants = - log_t(1.0 / logt_partition, t) + mu

    return normalization_constants

def compute_normalization_binary_search(activations, t, num_iters):

    """Returns the normalization value for each example (t < 1.0).
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature 2 (< 1.0 for finite support).
      num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    mu, _ = torch.max(activations, -1, keepdim=True)
    normalized_activations = activations - mu

    effective_dim = \
        torch.sum(
                (normalized_activations > -1.0 / (1.0-t)).to(torch.int32),
            dim=-1, keepdim=True).to(activations.dtype)

    shape_partition = activations.shape[:-1] + (1,)
    lower = torch.zeros(shape_partition, dtype=activations.dtype, device=activations.device)
    upper = -log_t(1.0/effective_dim, t) * torch.ones_like(lower)

    for _ in range(num_iters):
        logt_partition = (upper + lower)/2.0
        sum_probs = torch.sum(
                exp_t(normalized_activations - logt_partition, t),
                dim=-1, keepdim=True)
        update = (sum_probs < 1.0).to(activations.dtype)
        lower = torch.reshape(
                lower * update + (1.0-update) * logt_partition,
                shape_partition)
        upper = torch.reshape(
                upper * (1.0 - update) + update * logt_partition,
                shape_partition)

    logt_partition = (upper + lower)/2.0
    return logt_partition + mu

class ComputeNormalization(torch.autograd.Function):
    """
    Class implementing custom backward pass for compute_normalization. See compute_normalization.
    """
    @staticmethod
    def forward(ctx, activations, t, num_iters):
        if t < 1.0:
            normalization_constants = compute_normalization_binary_search(activations, t, num_iters)
        else:
            normalization_constants = compute_normalization_fixed_point(activations, t, num_iters)

        ctx.save_for_backward(activations, normalization_constants)
        ctx.t=t
        return normalization_constants

    @staticmethod
    def backward(ctx, grad_output):
        activations, normalization_constants = ctx.saved_tensors
        t = ctx.t
        normalized_activations = activations - normalization_constants 
        probabilities = exp_t(normalized_activations, t)
        escorts = probabilities.pow(t)
        escorts = escorts / escorts.sum(dim=-1, keepdim=True)
        grad_input = escorts * grad_output
        
        return grad_input, None, None

def compute_normalization(activations, t, num_iters=5):
    """Returns the normalization value for each example. 
    Backward pass is implemented.
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
      num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """
    return ComputeNormalization.apply(activations, t, num_iters)

def tempered_sigmoid(activations, t, num_iters = 5):
    """Tempered sigmoid function.
    Args:
      activations: Activations for the positive class for binary classification.
      t: Temperature tensor > 0.0.
      num_iters: Number of iterations to run the method.
    Returns:
      A probabilities tensor.
    """
    internal_activations = torch.stack([activations,
        torch.zeros_like(activations)],
        dim=-1)
    internal_probabilities = tempered_softmax(internal_activations, t, num_iters)
    return internal_probabilities[..., 0]


def tempered_softmax(activations, t, num_iters=5):
    """Tempered softmax function.
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature > 1.0.
      num_iters: Number of iterations to run the method.
    Returns:
      A probabilities tensor.
    """
    if t == 1.0:
        return activations.softmax(dim=-1)

    normalization_constants = compute_normalization(activations, t, num_iters)
    return exp_t(activations - normalization_constants, t)

def bi_tempered_binary_logistic_loss(activations,
        labels,
        t1,
        t2,
        label_smoothing = 0.0,
        num_iters=5,
        reduction='mean'):

    """Bi-Tempered binary logistic loss.
    Args:
      activations: A tensor containing activations for class 1.
      labels: A tensor with shape as activations, containing probabilities for class 1
      t1: Temperature 1 (< 1.0 for boundedness).
      t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
      label_smoothing: Label smoothing
      num_iters: Number of iterations to run the method.
    Returns:
      A loss tensor.
    """
    internal_activations = torch.stack([activations,
        torch.zeros_like(activations)],
        dim=-1)
    internal_labels = torch.stack([labels.to(activations.dtype),
        1.0 - labels.to(activations.dtype)],
        dim=-1)
    return bi_tempered_logistic_loss(internal_activations, 
            internal_labels,
            t1,
            t2,
            label_smoothing = label_smoothing,
            num_iters = num_iters,
            reduction = reduction)

def bi_tempered_logistic_loss(activations,
        labels,
        t1,
        t2,
        label_smoothing=0.0,
        num_iters=5,
        reduction = 'mean'):

    """Bi-Tempered Logistic Loss.
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      labels: A tensor with shape and dtype as activations (onehot), 
        or a long tensor of one dimension less than activations (pytorch standard)
      t1: Temperature 1 (< 1.0 for boundedness).
      t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
      label_smoothing: Label smoothing parameter between [0, 1). Default 0.0.
      num_iters: Number of iterations to run the method. Default 5.
      reduction: ``'none'`` | ``'mean'`` | ``'sum'``. Default ``'mean'``.
        ``'none'``: No reduction is applied, return shape is shape of
        activations without the last dimension.
        ``'mean'``: Loss is averaged over minibatch. Return shape (1,)
        ``'sum'``: Loss is summed over minibatch. Return shape (1,)
    Returns:
      A loss tensor.
    """

    if len(labels.shape)<len(activations.shape): #not one-hot
        labels_onehot = torch.zeros_like(activations)
        labels_onehot.scatter_(1, labels[..., None], 1)
    else:
        labels_onehot = labels

    if label_smoothing > 0:
        num_classes = labels_onehot.shape[-1]
        labels_onehot = ( 1 - label_smoothing * num_classes / (num_classes - 1) ) \
                * labels_onehot + \
                label_smoothing / (num_classes - 1)

    probabilities = tempered_softmax(activations, t2, num_iters)

    loss_values = labels_onehot * log_t(labels_onehot + 1e-10, t1) \
            - labels_onehot * log_t(probabilities, t1) \
            - labels_onehot.pow(2.0 - t1) / (2.0 - t1) \
            + probabilities.pow(2.0 - t1) / (2.0 - t1)
    loss_values = loss_values.sum(dim = -1) #sum over classes

    if reduction == 'none':
        return loss_values
    if reduction == 'sum':
        return loss_values.sum()
    if reduction == 'mean':
        return loss_values.mean()


[**Taylor Cross Entropy Loss Implementation**](https://www.kaggle.com/yerramvarun/cassava-taylorce-loss-label-smoothing-combo)

Discussion : https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/209782

In [ ]:


# implementations reference - https://github.com/CoinCheung/pytorch-loss/blob/master/pytorch_loss/taylor_softmax.py
# paper - https://www.ijcai.org/Proceedings/2020/0305.pdf

class TaylorSoftmax(nn.Module):

    def __init__(self, dim=1, n=2):
        super(TaylorSoftmax, self).__init__()
        assert n % 2 == 0
        self.dim = dim
        self.n = n

    def forward(self, x):
        
        fn = torch.ones_like(x)
        denor = 1.
        for i in range(1, self.n+1):
            denor *= i
            fn = fn + x.pow(i) / denor
        out = fn / fn.sum(dim=self.dim, keepdims=True)
        return out

class LabelSmoothingLoss(nn.Module):

    def __init__(self, classes, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        """Taylor Softmax and log are already applied on the logits"""
        #pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad(): 
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    

class TaylorCrossEntropyLoss(nn.Module):

    def __init__(self, n=2, ignore_index=-1, reduction='mean', smoothing=0.2):
        super(TaylorCrossEntropyLoss, self).__init__()
        assert n % 2 == 0
        self.taylor_softmax = TaylorSoftmax(dim=1, n=n)
        self.reduction = reduction
        self.ignore_index = ignore_index
        self.lab_smooth = LabelSmoothingLoss(out_dim, smoothing=smoothing)

    def forward(self, logits, labels):

        log_probs = self.taylor_softmax(logits).log()
        loss = F.nll_loss(log_probs, labels, reduction=self.reduction,
                ignore_index=self.ignore_index)
        #loss = self.lab_smooth(log_probs, labels)
        return loss



In [ ]:
criterion = TaylorCrossEntropyLoss(n=2, smoothing = smoothing )

# SymmetricCrossEntropy Loss

In [ ]:
import torch
import torch.nn.functional as F
#https://github.com/HanxunH/SCELoss-Reproduce/blob/master/loss.py

class SymmetricCrossEntropy(torch.nn.Module):
    def __init__(self, alpha = 0.1, beta=1.0, num_classes=5):
        super(SymmetricCrossEntropy, self).__init__()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.alpha = alpha
        self.beta = beta
        self.num_classes = num_classes
        self.cross_entropy = torch.nn.CrossEntropyLoss()

    def forward(self, pred, labels):
        # CCE
        ce = self.cross_entropy(pred, labels)

        # RCE
        pred = F.softmax(pred, dim=1)
        pred = torch.clamp(pred, min=1e-7, max=1.0)
        label_one_hot = torch.nn.functional.one_hot(labels, self.num_classes).float().to(self.device)
        label_one_hot = torch.clamp(label_one_hot, min=1e-4, max=1.0)
        rce = (-1*torch.sum(pred * torch.log(label_one_hot), dim=1))

        # Loss
        loss = self.alpha * ce + self.beta * rce.mean()
        return loss

# Train and eval function


* when we will call **train_epoch()** function, we will be training our model for 1 epoch every time using the train_loader that we are passing to this function for 8 core tpu training,from train_loader we are using batch images and labels and later feeding them to our model,calculating loss,doing backpropagation,taking optimizer step and returning training loss for each epoch

* **val_epoch()** function uses our validation dataloader that we are passing to this function for 8 core training on tpu.it calculates validation loss and validation accuracy using our validation dataset and returns both validation accuracy and validation loss, while calculatin validation accuracy we should do this : accuracy = xm.mesh_reduce('test_accuracy', acc, np.mean),if you don't do this then you won't be able to save best weight file for each fold and xm.save() will hang for forever(i made this silly mistake few days ago while working on this kernel,so keep this in mind)

NOTE : Latest Torch nightly and version 1.7 has some issue with  softmax i guess,so i couldn't make qishen ha san's val_epoch() working on tpu. i changed val_epoch() function code after this [discussion](https://github.com/pytorch/xla/issues/2712)  and it is working fine now

In [ ]:
def train_epoch(model,device,loader, optimizer):
 
    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        logits = model(data)
        #lossfn = SymmetricCrossEntropy()
        #loss = lossfn(logits, target)
        loss = bi_tempered_logistic_loss(logits, target, t1=t1, t2=t2, label_smoothing=smoothing)
            
        loss.backward()
        xm.optimizer_step(optimizer)
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        #bar.set_description('loss: %.5f' % (loss_np))
    return train_loss

def val_epoch(model,device,loader, get_output=False):
    model.eval()

    t = time.time()

    val_loss = []
    sample_num = 0
    image_preds_all = []
    image_targets_all = []

    pbar = tqdm(enumerate(loader), total=len(loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        #lossfn = SymmetricCrossEntropy()
        #loss = lossfn(image_preds, image_labels)
        loss = bi_tempered_logistic_loss(image_preds, image_labels, t1=t1, t2=t2, label_smoothing = smoothing)
           
        val_loss.append(loss.detach().cpu().numpy())
        
        sample_num += image_labels.shape[0]  

       
    val_loss = np.mean(val_loss)
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    acc = (image_preds_all==image_targets_all).mean()

    accuracy = xm.mesh_reduce('test_accuracy', acc, np.mean)
    xm.master_print("Validation Accuracy = ",accuracy)

    return val_loss,accuracy
    


# Cassava Model

in previous versions of this kernel i tried efficientnet lite models and they performer poorly,couldn't cross 75% validation accuracy,in my previous notebook [ViT - Pytorch xla (TPU) for leaf disease](https://www.kaggle.com/mobassir/vit-pytorch-xla-tpu-for-leaf-disease?scriptVersionId=51559451) i tried ViT base model which performed well so here in this notebook will try **vit_large_patch16_384** model now.

![](https://www.programmersought.com/images/83/cc5dacb45f0ea361e032b940ca8ed5c3.png)

# How it works?

[Visual Transformers (ViT)](https://pypi.org/project/pretrained-vit-pytorch/) are a straightforward application of the transformer architecture to image classification. Even in computer vision, it seems, attention is all you need.

The ViT architecture works as follows: (1) it considers an image as a 1-dimensional sequence of patches, (2) it prepends a classification token to the sequence, (3) it passes these patches through a transformer encoder (like BERT), (4) it passes the first token of the output of the transformer through a small MLP to obtain the classification logits. ViT is trained on a large-scale dataset (ImageNet-21k) with a huge amount of compute.

![](https://warehouse-camo.ingress.cmh1.psfhosted.org/49345a5e2e5aa356f28f4bbb54b89fccc5505603/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f676f6f676c652d72657365617263682f766973696f6e5f7472616e73666f726d65722f6d61737465722f666967757265312e706e67)

In [ ]:
model = cassavamodel(net_type, n_class=out_dim)

# Scheduler

In [ ]:

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]


init_lr = init_lr * xm.xrt_world_size()

optimizer = optim.Adam(model.parameters(), lr=init_lr)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)



lrs = []
for epoch in range(1, n_epochs+1):
    scheduler_warmup.step(epoch-1)
    lrs.append(optimizer.param_groups[0]["lr"])
rcParams['figure.figsize'] = 20,3
plt.plot(lrs)

# prepare Dataloader and Run Training


* unfortunately torch xla can't use our **dataset_train and dataset_valid** (that we are preparing below using the **LEAFDataset()** class that we have defined above) directly for 8 core training and that's why we use **DistributedSampler()** for preparing train_sampler and valid_sampler that will appropriately distribute the dataset across the 8 cores.

* using train_sampler and valid_sampler we finally create our train_loader and valid_loader that we will use for 8 core training on pytorch tpu by passing them in train_epoch() and val_epoch() function respectively

# Layer freezing in transfer learning

we will not freeze all layers like @piantic did [here](https://www.kaggle.com/piantic/how-to-finetuning-models-pytorch-xla-tpu), instead we will [TRAIN SOME LAYERS AND LEAVE OTHER FROZEN](https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/2)

The basic idea is that all models have a function model.children() which returns it’s layers. Within each layer, there are parameters (or weights), which can be obtained using .param() on any children (i.e. layer). Now, every parameter has an attribute called requires_grad which is by default True. True means it will be backpropagrated and hence to freeze a layer you need to set requires_grad to False for all parameters of a layer.inside epoch loop we are freezing  first 3 layers in the total N layers of ViT large model. Hope this helps!

![](https://qph.fs.quoracdn.net/main-qimg-96376d794775a37a272dac2a7a38f29e)

In [ ]:
#model = cassavamodel(net_type, n_class=out_dim)

def train_model():
    device = xm.xla_device()
    #global model
    #model = xm.send_cpu_data_to_device(model, device)
    model.to(device)
    for fold in range(folds):
        if(fold != 3):
            continue

        optimizer = optim.Adam(model.parameters(), lr=init_lr/warmup_factor)
        scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
        scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

        train_idx = np.where((df_train['fold'] != fold))[0]
        valid_idx = np.where((df_train['fold'] == fold))[0]
   
        df_this  = df_train.loc[train_idx]
        df_valid = df_train.loc[valid_idx]

        dataset_train = LEAFDataset(df_this , transforms=transforms_train)
        dataset_valid = LEAFDataset(df_valid, transforms=transforms_val)

        train_sampler = torch.utils.data.distributed.DistributedSampler(
            dataset_train,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True)


        valid_sampler = torch.utils.data.distributed.DistributedSampler(
            dataset_valid,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False,
            )


        train_loader = torch.utils.data.DataLoader(
            dataset=dataset_train,
            batch_size=batch_size,
            sampler=train_sampler,
            drop_last=True,
            num_workers=num_workers,
        )

        valid_loader = torch.utils.data.DataLoader(
            dataset=dataset_valid, 
            batch_size=batch_size,
            sampler=valid_sampler,
            drop_last=True,
            num_workers=num_workers,
        )

        acc_max = 0.
        count = 0
        for epoch in range(1, n_epochs+1):
            scheduler.step(epoch-1)
            
            #https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/32
            
            if epoch < 4:
                # freeze backbone layers
                for param in model.parameters():
                    count +=1
                    if count < 4: #freezing first 3 layers
                        param.requires_grad = False

            else:
                for param in model.parameters():
                    param.requires_grad = True

            para_loader = pl.ParallelLoader(train_loader, [device])

            train_loss = train_epoch(model,device,para_loader.per_device_loader(device), optimizer)
            del para_loader
            gc.collect()
        
            para_loader = pl.ParallelLoader(valid_loader, [device])

            val_loss, acc = val_epoch(model,device,para_loader.per_device_loader(device))
            
            del para_loader
            gc.collect()
            
            #xm.master_print(device)
         

            content = time.ctime() + ' ' + f'FOLD -> {fold} --> Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, acc: {(acc):.5f}'

            with open(f'log_{kernel_type}.txt', 'a') as appender:
                appender.write(content + '\n')
            

            #xm.save(model.state_dict(), os.path.join(f'{kernel_type}_fold{fold}_epoch_{epoch}_val_acc_{acc}.pth'))
        
            best_file = f'{kernel_type}_best_fold.pth'

            if acc > acc_max:
                xm.save(model.state_dict(), os.path.join(best_file))
                #xser.save(model.state_dict(), os.path.join(best_file))
                acc_max = acc
            
        #xm.save(model.state_dict(), os.path.join(f'{kernel_type}_fold{fold}.pth'))    



# Start training processes

**In order to train the model, we need to spawn the training processes on each of the TPU cores.we will call train_model() function during spawn call**

In [ ]:
%%time

def _mp_fn(rank, flags):
    global acc_list
    torch.set_default_tensor_type('torch.FloatTensor')
    res = train_model()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

cv 0.9 in previous versions of this kernel

# Full Training Log

In [ ]:
f = open(f'./log_{kernel_type}.txt', "r")

print(f.read())

# Thank you for your time!

